<a href="https://colab.research.google.com/github/farmountain/SmartGlass-AI-Agent/blob/main/colab_notebooks/Session10_Caching_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Session 10: Caching & Optimization
**Goal:** Improve performance of audio/image/text pipelines using caching and inference optimization.

We'll learn how to:
- Cache model outputs using `joblib`, `pickle`, and `diskcache`
- Use `TorchScript` or `ONNX` to optimize model inference
- Apply audio/image/text pre-processing caching techniques

In [ ]:
# ✅ Install dependencies
!pip install -q torch torchvision torchaudio transformers openai-whisper onnxruntime diskcache joblib gtts

In [ ]:
# 2. Generate Audio
from gtts import gTTS

text_to_speak = "Hello, this is an example audio file generated by gTTS."
tts = gTTS(text=text_to_speak, lang='en')
AUDIO_PATH = 'example.wav'
tts.save(AUDIO_PATH)

print(f'Generated audio file: {AUDIO_PATH}')

In [ ]:
# ✅ Load Whisper and cache transcription results
import whisper, os
from gtts import gTTS
from diskcache import Cache

# Generate a sample audio if not exists
AUDIO_PATH = 'example.wav'
if not os.path.exists(AUDIO_PATH):
    print(f"Audio file '{AUDIO_PATH}' not found. Generating with gTTS...")
    tts = gTTS(text="Hello, this is an example audio file generated by gTTS.", lang='en')
    tts.save(AUDIO_PATH)
    print(f'Generated audio file: {AUDIO_PATH}')

# Load Whisper model and initialize cache
model = whisper.load_model('tiny')
cache = Cache('./whisper_cache')

def cached_transcribe(audio_path):
    if audio_path in cache:
        print('📦 Using cached transcription')
        return cache[audio_path]
    result = model.transcribe(audio_path)
    cache[audio_path] = result['text']
    return result['text']

text = cached_transcribe(AUDIO_PATH)
print('🗣️ Text:', text)


In [ ]:
import torch
from transformers import GPT2Model, GPT2Tokenizer

# Load model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')
model.eval()

# Wrapper class to return only last_hidden_state
class GPT2Wrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids):
        output = self.model(input_ids)
        return output.last_hidden_state

# Wrap the model
wrapped_model = GPT2Wrapper(model)

# Sample input
example = tokenizer("Hello from smart glasses!", return_tensors='pt')

# Trace the model
traced_model = torch.jit.trace(wrapped_model, (example['input_ids'],))
traced_model.save("gpt2_traced.pt")

print('🚀 Traced GPT-2 model saved ✅')


In [ ]:
# ✅ Cache image features using CLIP
from PIL import Image
from transformers import CLIPModel, CLIPProcessor
import torch
import os
import joblib
from google.colab import files

clip_model = CLIPModel.from_pretrained('openai/clip-vit-base-patch32')
clip_processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32')
clip_model.eval()

# Allow user to upload an image file
uploaded = files.upload()

if uploaded:
    # Assuming only one file is uploaded
    image_filename = list(uploaded.keys())[0]
    print(f"Uploaded file: {image_filename}")

    try:
        img = Image.open(image_filename)
        input = clip_processor(images=img, return_tensors='pt')

        # Use a filename based on the uploaded file for caching
        cache_filename = f'{os.path.splitext(image_filename)[0]}_clip_features.pkl'

        if os.path.exists(cache_filename):
            print('📦 Loading cached features')
            features = joblib.load(cache_filename)
        else:
            with torch.no_grad():
                features = clip_model.get_image_features(**input)
            joblib.dump(features, cache_filename)
        print('🧠 Features:', features.shape)

    except FileNotFoundError:
        print(f"Error: File not found after upload: {image_filename}")
    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print("No file uploaded.")